In [26]:
import requests
import asyncio
import json
import websockets

BASE_URL = "http://localhost:8000" 

def add_user(firebase_uid: str, name: str):
    user_model = {"firebase_uid": firebase_uid, "name": name}
    response = requests.post(f"{BASE_URL}/users/", json=user_model)
    return response.status_code, response.json()

def get_users():
    response = requests.get(f"{BASE_URL}/users/")
    return response.status_code, response.json()

def get_user(id: str):
    response = requests.get(f"{BASE_URL}/users/{id}")
    return response.status_code, response.json()

def update_user(id: str, name: str):
    response = requests.put(f"{BASE_URL}/users/{id}", json={"name": name})
    return response.status_code, response.json()

def delete_user(id: str):
    response = requests.delete(f"{BASE_URL}/users/{id}")
    return response.status_code

def get_games():
    response = requests.get(f"{BASE_URL}/games/")
    return response.status_code, response.json()

def create_game(name: str, game_type: str):
    response = requests.post(f"{BASE_URL}/games/", json={"name": name, "type": game_type})
    return response.status_code, response.json()

def add_user_to_game(game_id: str, user_id: str):
    response = requests.patch(f"{BASE_URL}/games/{game_id}/add_user/{user_id}")
    return response.status_code, response.json()

def remove_user_from_game(game_id: str, user_id: str):
    response = requests.patch(f"{BASE_URL}/games/{game_id}/remove_user/{user_id}")
    return response.status_code, response.json()

def start_game(game_id: str):
    response = requests.patch(f"{BASE_URL}/games/{game_id}/start")
    return response.status_code

def delete_game(game_id: str):
    response = requests.delete(f"{BASE_URL}/games/{game_id}")
    return response.status_code

async def listen(game_id):
    uri = f"ws://localhost:8000/ws/game/{game_id}"  # Replace with your server URL
    async with websockets.connect(uri) as websocket:
        print("Connected to WebSocket")
        while True:
            message = await websocket.recv()
            data = json.loads(message)
            print(data)

def get_active_games():
    response = requests.get(f"{BASE_URL}/games/active")
    return response.status_code, response.json()

def get_active_game_debug(game_id: str):
    response = requests.get(f"{BASE_URL}/games/active/{game_id}/debug")
    return response.status_code, response.json()

def play_turn(game_id: str, player_id: str, turn_type: int, sender_id: str, receiver_id: str, cards: list[str]):
    suit_dict = {"J":0, "C":1, "D":2, "H":3, "S":4}
    rank_dict = {"0":0, "A":1, "2":2, "3":3, "4":4, "5":5, "6":6, "7":7, "8":8, "9":9, "T":10, "J":11, "Q":12, "K":13}
    turn = {"player":player_id, "transactions":[{"sender":sender_id, "receiver":receiver_id, "card":{"rank":rank_dict[card[0]], "suit":suit_dict[card[1]]}} for card in cards], "type":turn_type}
    response = requests.patch(f"{BASE_URL}/games/{game_id}/play", json=turn)
    return response.status_code
            

In [27]:
print(add_user("12345test","ethantest"))

(409, {'detail': 'User with Firebase UID 12345test already exists.'})


In [28]:
print(get_users())

(200, {'users': [{'id': '681fdb709d14dcc91d530575', 'firebase_uid': 'dJ84cRICX2MtkzOi9ceaK333QPQ2', 'name': 'lordboba', 'games': 0, 'wins': 0, 'username_set': True}, {'id': '6826db221601e717a6843e86', 'firebase_uid': 'abc123firebase', 'name': 'cool_user', 'games': 0, 'wins': 0, 'username_set': True}, {'id': '6826dce6e8bce8f5a061dfdc', 'firebase_uid': 'test-uid-123', 'name': None, 'games': 0, 'wins': 0, 'username_set': False}, {'id': '6826dd2b1601e717a6843e87', 'firebase_uid': 'hkrwtgoIy7bJRttAASiFyLCtkUs1', 'name': 'ethan', 'games': 0, 'wins': 0, 'username_set': True}, {'id': '68277f750a9dcc37ba836316', 'firebase_uid': 'abc456firebase', 'name': 'cool_user2', 'games': 0, 'wins': 0, 'username_set': True}, {'id': '6830065c317d37d03524150f', 'firebase_uid': '12345test', 'name': 'ethantest', 'games': 0, 'wins': 0, 'username_set': True}]})


In [29]:
print(get_user("682e907f9ead523f1a672ee0"))

(404, {'detail': 'User 682e907f9ead523f1a672ee0 not found'})


In [30]:
print(update_user("682e907f9ead523f1a672ee0","ethantest2"))

(404, {'detail': 'User 682e907f9ead523f1a672ee0 not found during update attempt.'})


In [31]:
print(delete_user("682e907f9ead523f1a672ee0"))

404


In [32]:
print(get_games())

(200, {'games': [{'id': '682ed1647e56583072af0a58', 'name': 'ethantestgame', 'type': 'simple', 'players': ['681fdb709d14dcc91d530575', '6826db221601e717a6843e86']}, {'id': '6830065c317d37d035241510', 'name': 'ethantestgame', 'type': 'simple', 'players': []}]})


In [33]:
print(create_game("ethantestgame", "simple"))

(200, {'id': '68300686317d37d035241512', 'name': 'ethantestgame', 'type': 'simple', 'players': []})


In [34]:
print(add_user_to_game("682ec12a14dfcbc81a10a3e2","681fdb709d14dcc91d530575"))

(404, {'detail': 'Game not found'})


In [35]:
print(remove_user_from_game("682e9cda9ead523f1a672ee1","681fdb709d14dcc91d530575"))

(404, {'detail': 'Game not found'})


In [36]:
print(start_game("682ec12a14dfcbc81a10a3e2"))

404


In [37]:
print(delete_game("682ec7a2ca86c0ff48611a0c"))

404


In [38]:
print(get_active_games())

(200, ['6830065c317d37d035241511'])


In [39]:
# Create simple game
id1 = "681fdb709d14dcc91d530575"
id2 = "6826db221601e717a6843e86"
_, game_dict = create_game("ethantestgame", "simple")
add_user_to_game(game_dict["id"],id1)
add_user_to_game(game_dict["id"],id2)
start_game(game_dict["id"])

204

In [40]:
print(get_active_game_debug(game_dict["id"]))

(200, ['AC', '2C', '3C', '4C', '5C', '6C', '7C', '8C', '9C', 'TC'])


In [41]:
asyncio.run(listen(game_dict["id"]))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [42]:
print(play_turn(game_dict["id"], id1, 0, id1, id2, ["AC", "9C"]))

204
